# MultiTree

The `toytree.MultiTree` class object is used to represent a collection of `ToyTree` objects and includes attributes and methods for describing this set or performing operations on it. Common examples of tree sets include bootstrap replicate samples or posterior distributions of sampled trees; common operations on sets of trees include *consensus tree inference*, computing discordance or distance statistics, and plotting tree grids or cloud trees.

In [1]:
import toytree

## Generating MultiTrees

MultiTree objects can be generated from a list of Toytrees or newick strings, or by parsing a file, url, or string of text that includes newick trees separated by newlines. The convenience function `toytree.mtree()` can be used to parse multitree input data similar to how the function `toytree.tree` is used to parse individual trees, and supports the same file formats.

### From tree data
Below is an example multi-newick string representing multiple trees as newick strings separated by newlines. You can create a MultiTree from this input data, entered as a string or filepath, by passing it to the `toytree.mtree()` convenience parsing function. Each tree will be parsed individually and stored as a list of `ToyTree` objects contained within a returned `MultiTree` object.

In [2]:
multinewick = """\
(((a:1,b:1):1,(d:1.5,e:1.5):0.5):1,c:3);
(((a:1,d:1):1,(b:1,e:1):1):1,c:3);
(((a:1.5,b:1.5):1,(d:1,e:1):1.5):1,c:3.5);
(((a:1.25,b:1.25):0.75,(d:1,e:1):1):1,c:3);
(((a:1,b:1):1,(d:1.5,e:1.5):0.5):1,c:3);
(((b:1,a:1):1,(d:1.5,e:1.5):0.5):2,c:4);
(((a:1.5,b:1.5):0.5,(d:1,e:1):1):1,c:3);
(((b:1.5,d:1.5):0.5,(a:1,e:1):1):1,c:3);
"""

In [3]:
# create an mtree from a string, list of strings, url, or file.
mtree1 = toytree.mtree(multinewick)
mtree1

<toytree.MultiTree ntrees=8>

### From a collection of trees
Similarly, you can create a `MultiTree` by providing a collection of `ToyTree` objects to the `toytree.mtree` function. Here we generate a list of 50 random coalescent trees and pass the list as input to create a new `MultiTree`.

In [4]:
# generate 50 random coalescent trees each with 6 tips
coaltrees = [toytree.rtree.coaltree(k=6) for i in range(50)]

In [5]:
# create a MultiTree from a list of ToyTrees
mtree2 = toytree.mtree(coaltrees)
mtree2

<toytree.MultiTree ntrees=50>

## Indexable and Iterable
One or more trees can be indexed or sliced from a `MultiTree`, and sequential trees can be accessed through iteration. The trees themselves are stored in the `.treelist` attribute of the `MultiTree` object as a list. This can be modified to remove, add, or reorder the trees. Several example operations are shown below for accessing one or more trees.

In [6]:
# get first tree
mtree1[0]

In [7]:
# get all trees
mtree1[:]

In [8]:
# slice the first three trees
mtree1[:3]

In [9]:
# iterate over ToyTrees in a MultiTree
for tree in mtree1:
    print(tree)

In [10]:
# re-arrange trees in the treelist to send the first to be last
mtree1.treelist = mtree1.treelist[1:] + [mtree1.treelist[0]]
mtree1[:]

## Attributes and types of tree sets
Most of the time `MultiTree` objects are used to hold a collection of trees that all share the same tip labels, such as a collection of bootstrap replicates. But, in other cases, a `MultiTree` could hold a collection of unrelated trees, in which case some of the built-in functions for comparing trees (such as consensus tree inference) will raise an error, but it still provides a useful container for drawing trees. These methods will raise a ToyTreeError when attempted if the tree set is a mixed collection of trees. The  `MultiTree` class contains several functions to quickly check attributes of the tree set to examine the number of trees, whether they share the same tip names, and whether the trees are rooted or ultrametric.

In [11]:
mtree1.ntrees

8

In [12]:
mtree1.all_tree_tip_labels_same()

True

In [13]:
mtree1.all_tree_topologies_same()

False

In [14]:
mtree1.all_tree_tips_aligned()

False

## Consensus trees
A majority-rule consensus tree is a tree that represents the most common non-conflicting splits among a set of input trees. In `toytree` this can be inferred from a `MultiTree` object using the `.get_consensus_tree`, which returns a `ToyTree` with the consensus topology on which several types of statistics have been stored to nodes. This can be useful in several contexts, but one of the simplest is to find the most common order of tip names among a set of trees, so that when you make a `MultiTree` plot you can order tips in the same way to more easily visualize variation among trees. There are many options and uses for consensus trees, check out the [Consensus Trees](/toytree/consensus-trees) documentation section for more details.

In the example below we get a consensus tree from the `mtree1` multitree object which contains 8 trees. The support values on this returned tree represent the proportion of trees in the input set that contain each clade in the consensus tree. By default, the returned tree is unrooted, since the input trees may vary in their rooting. Here the trees exhibit variation in the relationships among `(a,b,d,e)` with `(a,b)` and `(d,e)` each occurring in 75% of trees.

In [15]:
# get a consensus tree 
ctree = mtree1.get_consensus_tree()

# plot the tree showing 'support' values
ctree.draw(
    ts='r', layout="unrooted", width=350, height=350,
    node_labels="support", node_sizes=14, node_as_edge_data=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="350.0px" viewBox="0 0 350.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb701332c16884b06865d750a9501177e"> 0.75 0.75 a b d e c

## Unique trees
Given a set of trees it is useful to be able to pull out just the unique topologies from the set. The function `get_unique_topologies()` returns a list of `(tree, int)` tuples from a `MultiTree` with each unique topology paired with its number of occurrences in the set. Note, this condenses all trees with the same topology into a single representative, using the first occurrence as the returned tree, thus branch length variation is not retained. See the `toytree.distance` subpackage for many additional methods for comparing trees and computing differences/distances between them. 

In [16]:
# get (tree, count) for each unique topology in the MultiTree
mtree1.get_unique_topologies()

[[<toytree.ToyTree at 0x7f03c5ab9270>, 6],
 [<toytree.ToyTree at 0x7f03c5ab97e0>, 1],
 [<toytree.ToyTree at 0x7f03c58ec190>, 1]]

## Drawing with MultiTrees
There are two main ways to draw groups of trees from `MultiTree` objects. The first is simply called `.draw()` and is used to plot multiple trees onto a shared canvas arranged into a grid. These trees can be arranged each on their own axes, or on shared axes to better compare scales. The second method is `.draw_cloud_tree`, which plots a set of trees over top of one another, usually at low opacity, as a way of examining discordance and variation among a set of trees. These are sometimes called cloud trees, densitrees, or other similar names. A simple example of each is shown below.

### Grid tree drawings
The `.draw()` function of a `MultiTree` returns multiple tree drawings layed out on a grid. The shape of the grid, and which trees from the tree set are drawn, can be set with the `shape` and `idxs` arguments. This also accepts all of the standard drawing arguments that can be used when drawing individual trees. See [Drawing Tree Grids](/toytree/drawing-tree-grids).

In [19]:
# draw a 2x4 grid of trees with 30px margins between them
mtree1.draw(ts='o', shape=(2, 4), width=600, height=300, margin=25, fixed_order=['c', 'b', 'e', 'a', 'd']);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1f8a2244cc02496895532bee7e489ae5"> a d b e c a b d e c a b d e c a b d e c b a d e c a b d e c b d a e c a b d e c

### Cloud tree drawings
It is sometimes even more informative to plot a number of trees on top of each other to visualize their discordance. These are sometimes called “densitree” plots, or here, “cloud tree plots”. See the documentation section on [Drawing Cloud Trees](#drawing-cloud-trees) for more details and examples.

In [27]:
# draw a cloud tree
mtree1.draw_cloud_tree(
    scale_bar=True,
    edge_style={
        "stroke-opacity": 0.1,
        "stroke-width": 3,
    },
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t71f4e467e42d4544bf6f3ed78040f08e"> a d b e c 3 2 1 0